# Capstone Project - The Battle of Neighborhoods

### Notebook for the battle of the Neighborhoods Capstone. We will be exploring the Foursquare API data and how to apply it to a business problem. 

## Business Problem

### We will be exploring the Washington D.C. area in order to assist in informing our client the best location for opening a high-end French restuarant. We will identify a location in D.C. that explores the area for other restaurants, other French restaurants, and the median income of the area and surrounding areas, based on median income data from Census. Our stakeholders include not only our client, but also the customers who will be eating there, and an exploration of available transportation in the area will also be explored.

## Data 

### The three following data sets will be used to arrive at a conclusino for the best location for a high-end Frech restaurant based on competition, median income, and available transportation in the area:

### Foursquare Data: We will use the Foursquare API to explore French restaurant locations in D.C. and pick a location that has little competition for out stakeholder (client).

### Census Data: We will use 2010 Census data in order to find the median income by Census block, and use this to find the areas with the highest income, so the restaurant can experience the highest traffic possible given the high price point of the food.

### DC Open Data Transporation: We will use the DC Open Data portal in order to find transportation in the area that will make it easier for one of our stakeholders (customers) to get there.

In [4]:
import requests 
import pandas as pd
import numpy as np 
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

import config

# Foursquare API info
CLIENT_ID = config.CLIENT_ID # your Foursquare ID
CLIENT_SECRET = config.CLIENT_SECRET # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30